## 0. Import packages

In [1]:
import pandas as pd
import gzip
import math
from tqdm import tqdm
from itertools import chain, combinations
from Association import Association
import numpy as np
tqdm.pandas() #for progres_apply etc.

## 1. Load data from previous step

In [2]:
import pickle
rules = pickle.load(open("pickle_dumps/article_rules.p", "rb"))
df_test = pickle.load(open("pickle_dumps/test_df.p", "rb"))
df_test = df_test.set_index("user_id").to_dict()["item_id"]

In [3]:
print(len(rules))
for i in range(10):
    if i < len(rules):
        print(rules[i])

6997716
Conf: 1.00	Supp: 50.00	 {736, 4, 167, 1576, 780, 750, 1839, 212, 1597} => {1328}
Conf: 1.00	Supp: 50.00	 {736, 167, 1576, 780, 750, 1839, 1328, 212, 1597} => {4}
Conf: 0.98	Supp: 50.00	 {736, 4, 167, 1576, 780, 750, 1328, 212, 1597} => {1839}
Conf: 0.98	Supp: 50.00	 {736, 4, 167, 1576, 750, 1839, 1328, 212, 1597} => {780}
Conf: 0.98	Supp: 50.00	 {4, 167, 1576, 780, 750, 1839, 1328, 212, 1597} => {736}
Conf: 0.96	Supp: 50.00	 {736, 4, 167, 1576, 780, 750, 1839, 1328, 212} => {1597}
Conf: 0.96	Supp: 50.00	 {736, 4, 167, 1576, 780, 750, 1839, 1328, 1597} => {212}
Conf: 0.96	Supp: 50.00	 {736, 4, 167, 1576, 780, 1839, 1328, 212, 1597} => {750}
Conf: 0.96	Supp: 50.00	 {736, 4, 167, 780, 750, 1839, 1328, 212, 1597} => {1576}
Conf: 0.96	Supp: 50.00	 {736, 4, 1576, 780, 750, 1839, 1328, 212, 1597} => {167}


In [4]:
class Recommender():
    def __init__(self, df_train, assos, df_test):
        pass
    
    def recommend(self, user_id, k):
        pass        

In [29]:
class Assos_Recommender(Recommender):
    def __init__(self, assos):
        # use popular items to fill unused space
        # TODO (df_train is used for this)
        
        self.assos = sorted(assos, key=lambda x: x.s, reverse=True)
        
#     def get_items(self, user_id):
#         return self.df_test[user_id]
    
    def recommend(self, user_items, k):
        recommendation = set()
        for a in self.assos:
            for l in a.left:
                # break if any element is not in the user session
                if l not in user_items:
                    break
                    
                # all elements are in the user session --> add RHS of rule to the recommendation
                r = list(a.right)[0]
                if r not in user_items:
                    recommendation.add(r)
                        
            if len(recommendation) >= k and k != -1:
                break
                
        return recommendation

In [6]:
def splitUser(user_items):
    splits = []
    
#     scores = cross_val_score(clf, X, y, cv=5)
    tmp = np.array_split(user_items, 4)
    
    for l in tmp:
        if len(l) > 0:
            splits.append(list(l))
    
    res = []
    for i in splits:
        join = []
        for j in splits:
            if i != j:
                join += j
        res.append((join, i))
                    
                

    return res

print(splitUser(range(10)))

[([3, 4, 5, 6, 7, 8, 9], [0, 1, 2]), ([0, 1, 2, 6, 7, 8, 9], [3, 4, 5]), ([0, 1, 2, 3, 4, 5, 8, 9], [6, 7]), ([0, 1, 2, 3, 4, 5, 6, 7], [8, 9])]


In [7]:
def hitrate(rc, k):
    hits = 0
    counts = 0
    avg_len = 0
    for user in tqdm(df_test):
        user_items = df_test[user]
        split_user = splitUser(user_items)
        for split in split_user:
            r = rc.recommend(split[0], k)
        
            avg_len += len(r)/len(split_user)
            
            for item in split[1]:
                if item in r:
                    hits += (item in r) / len(split_user)
                    break
                
        counts += 1
    
    print("avg len: {:f}".format(avg_len / len(df_test)))
    if k == -1:
        print("HR\t{:.5f}".format((hits / counts)))
    else:
        print("HR@{}\t{:.5f}".format(k, (hits / counts)))
    return hits / counts

In [31]:
rc = Assos_Recommender(rules)
print("go")
hr = hitrate(rc, 10)

go


100%|██████████| 101/101 [05:55<00:00,  3.52s/it]

avg len: 20.000000
HR@20	0.33416


In [ ]:
print(hr)

# ??. Store the association rules as pickle file for further use

In [ ]:
# import pickle
# pickle.dump(rules, open("pickle_dumps/article_rules.p", "wb"))